<a href="https://colab.research.google.com/github/TranQuocHuy83/RAG/blob/colab-dev/notebooks/en/rag_evaluation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Đánh giá RAG**
_Tác giả: [Aymeric Roucher](https://huggingface.co/m-ric)_

Sổ tay này minh họa cách bạn có thể đánh giá hệ thống RAG (Retrieval Augmented Generation-Thế hệ tăng cường truy suất) của mình bằng cách xây dựng một tập dữ liệu đánh giá tổng hợp và sử dụng LLM làm công cụ đánh giá để tính toán độ chính xác của hệ thống.

Để tìm hiểu thêm về RAG, bạn có thể xem [sổ tay khác này](rag_zephyr_langchain)!

Hệ thống RAG rất phức tạp: đây là sơ đồ RAG, trong đó chúng tôi đã đánh dấu màu xanh lam tất cả các khả năng cải tiến hệ thống:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">

Việc triển khai bất kỳ cải tiến nào trong số này đều có thể mang lại hiệu suất tăng đáng kể; nhưng việc thay đổi bất cứ điều gì cũng vô ích nếu bạn không thể theo dõi tác động của những thay đổi đó đến hiệu suất của hệ thống!

Vậy hãy cùng xem cách đánh giá hệ thống RAG của chúng ta.

### **Đánh giá hiệu suất của RAG**

Vì có rất nhiều thành phần cần điều chỉnh và ảnh hưởng lớn đến hiệu suất, việc đánh giá hiệu năng của hệ thống RAG là rất quan trọng.

Để xây dựng quy trình đánh giá, chúng ta cần:
1. Một tập dữ liệu đánh giá với các cặp câu hỏi - câu trả lời (cặp QA)
2. Một bộ đánh giá để tính toán độ chính xác của hệ thống trên tập dữ liệu đánh giá ở trên.

➡️ Hóa ra, chúng ta có thể sử dụng LLM để hỗ trợ trong suốt quá trình!

1. Tập dữ liệu đánh giá sẽ được tạo ra một cách tổng hợp bởi một LLM 🤖, và các câu hỏi sẽ được lọc ra bởi các LLM khác 🤖
2. Một tác nhân [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) 🤖 sau đó sẽ thực hiện đánh giá trên tập dữ liệu tổng hợp này.

__Hãy cùng tìm hiểu và bắt đầu xây dựng quy trình đánh giá của chúng ta!__ Đầu tiên, chúng ta cài đặt các phụ thuộc mô hình cần thiết.

In [3]:
!pip install -q torch transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille
!pip install -U langchain-text-splitters langchain-community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 140.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 k

In [4]:
# %reload_ext autoreload
# %autoreload 2

In [5]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None) #Hiển thị trọn vẹn nội dung văn bản

In [6]:
from huggingface_hub import notebook_login

notebook_login()

### **Load your knowledge base**

In [7]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [8]:
print(ds)

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})


In [9]:
import pandas as pd

# Chuyển 5 dòng đầu tiên sang dạng bảng
df_view = pd.DataFrame(ds.select(range(5)))
display(df_view)

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# **1. Xây dựng bộ dữ liệu tổng hợp để đánh giá**
Trước tiên, chúng ta xây dựng một bộ dữ liệu tổng hợp gồm các câu hỏi và ngữ cảnh liên quan. Phương pháp là lấy các phần tử từ cơ sở tri thức của chúng ta, và yêu cầu một LLM tạo ra các câu hỏi dựa trên các tài liệu này.

Sau đó, chúng ta thiết lập các tác nhân LLM khác để hoạt động như các bộ lọc chất lượng cho các cặp câu hỏi-trả lời được tạo ra: mỗi tác nhân sẽ hoạt động như một bộ lọc cho một lỗi cụ thể.

### **1.1. Prepare source documents**

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = [] # danh sách văn bản ngắn --> chuẩn bị chuyển thành vector
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])
# kiểm tra số lượng văn bản
print(len(docs_processed))

  0%|          | 0/2647 [00:00<?, ?it/s]

28308


### **1.2. Thiết lập tác nhân để tạo câu hỏi**

Chúng tôi sử dụng [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) để tạo cặp câu hỏi QA vì nó có hiệu suất tuyệt vời trên các bảng xếp hạng như [Chatbot Arena](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard).

In [28]:
from huggingface_hub import InferenceClient

# Khai báo mô hình
# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "HuggingFaceH4/zephyr-7b-beta"
repo_id = "meta-llama/Llama-3.2-3B-Instruct"

llm_client = InferenceClient(model=repo_id, timeout=120) # công cụ gửi yêu cầu lên server


def call_llm(inference_client: InferenceClient, prompt: str):
  # Nhận:
  # inference_client → kết nối server
  # prompt → nội dung người dùng

    # Sử dụng chat_completion thay vì text_generation
    messages1 = [{"role": "user", "content": prompt}]

    response = inference_client.chat_completion(
        messages=messages1,
        max_tokens=200,
        temperature=0.1
    )
    return response.choices[0].message.content

# Chạy thử nghiệm
try:
    result = call_llm(llm_client, "Việt Nam có bao nhiêu tỉnh/thành?")
    print(result)
except Exception as e:
    print(f"Lỗi: {e}")

Việt Nam hiện có 63 tỉnh, thành phố.


In [29]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [32]:
# 2. Thử nghiệm với một đoạn văn bản thật trong docs_processed
import random

# Lấy ngẫu nhiên 1 đoạn trong số 28.308 đoạn đã chia
random_idx = random.randint(0, len(docs_processed)-1)
sample_context = docs_processed[random_idx].page_content

# Gửi cho AI
print(f"--- ĐOẠN VĂN GỐC ---\n{sample_context}\n")
result = call_llm(llm_client, QA_generation_prompt.format(context=sample_context))
print(f"--- AI TỰ ĐẶT CÂU HỎI ---\n{result}")

--- ĐOẠN VĂN GỐC ---
The old code: [link](https://github.com/huggingface/transformers/blob/ca2a55e9dfb245527b5e1c954fec6ffbb7aef07b/src/transformers/models/bloom/modeling_bloom.py#L94-L132)
The new code: [link](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bloom/modeling_bloom.py#L86-L127)

This is a 10x speedup and the latest version includes padding too!
Since this step is only computed once, the actual speed is not important
but overall reducing the number of operations and tensor creation is a good direction.

Other parts come out more clearly when you start [profiling](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html) and we used quite extensively the [tensorboard extension](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html)

This provides this sort of image which give insights:

--- AI TỰ ĐẶT CÂU HỎI ---
Factoid question: What is the main change in the Bloom model's computation in the latest version com

Bây giờ chúng ta hãy tạo các cặp kiểm thử chất lượng (QA).

Trong ví dụ này, chúng ta chỉ tạo 5 cặp QA và sẽ tải phần còn lại từ Hub.

Nhưng đối với cơ sở tri thức cụ thể của bạn, giả sử bạn muốn có ít nhất khoảng 100 mẫu thử nghiệm, và tính đến việc chúng ta sẽ lọc bỏ khoảng một nửa số này bằng các tác nhân phê bình sau này, bạn nên tạo nhiều hơn, khoảng >200 mẫu.

In [33]:
import random
import re

N_GENERATIONS = 5  # We intentionally generate only 10 QA couples here for cost and time considerations

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    raw_output = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )

    try:
        # Regex này sẽ tìm nội dung nằm sau 'Question:' và 'Answer:' bất kể viết hoa hay thường
        question_match = re.search(r"(?:Question|Factoid question):\s*(.*)", raw_output, re.IGNORECASE)
        answer_match = re.search(r"Answer:\s*(.*)", raw_output, re.IGNORECASE)

        if question_match and answer_match:
            question = question_match.group(1).split("Answer:")[0].strip()
            answer = answer_match.group(1).strip()

            # Chỉ lưu nếu câu trả lời không quá dài và không bị rỗng
            if 10 < len(answer) < 500:
                outputs.append({
                    "context": sampled_context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": sampled_context.metadata["source"],
                })
    except Exception as e:
        continue

Generating 5 QA couples...


  0%|          | 0/5 [00:00<?, ?it/s]

In [34]:
print(outputs)

[{'context': '## Eager data processing and lazy data processing\n\nWhen you process a [`Dataset`] object using [`Dataset.map`], the entire dataset is processed immediately and returned.\nThis is similar to how `pandas` works for example.\n\n```python\nmy_dataset = my_dataset.map(process_fn)  # process_fn is applied on all the examples of the dataset\nprint(my_dataset[0])\n```\n\nOn the other hand, due to the "lazy" nature of an `IterableDataset`, calling [`IterableDataset.map`] does not apply your `map` function over the full dataset.\nInstead, your `map` function is applied on-the-fly.\n\nBecause of that, you can chain multiple processing steps and they will all run at once when you start iterating over the dataset:\n\n```python\nmy_iterable_dataset = my_iterable_dataset.map(process_fn_1)\nmy_iterable_dataset = my_iterable_dataset.filter(filter_fn)\nmy_iterable_dataset = my_iterable_dataset.map(process_fn_2)', 'question': 'What is the difference between `Dataset.map` and `IterableData

In [35]:
# Chuyển thành DataFrame để dễ quan sát
display(pd.DataFrame(outputs).head(2))

,context,question,answer,source_doc
0,"## Eager data processing and lazy data processing\n\nWhen you process a [`Dataset`] object using [`Dataset.map`], the entire dataset is processed immediately and returned.\nThis is similar to how `pandas` works for example.\n\n```python\nmy_dataset = my_dataset.map(process_fn) # process_fn is applied on all the examples of the dataset\nprint(my_dataset[0])\n```\n\nOn the other hand, due to the ""lazy"" nature of an `IterableDataset`, calling [`IterableDataset.map`] does not apply your `map` function over the full dataset.\nInstead, your `map` function is applied on-the-fly.\n\nBecause of that, you can chain multiple processing steps and they will all run at once when you start iterating over the dataset:\n\n```python\nmy_iterable_dataset = my_iterable_dataset.map(process_fn_1)\nmy_iterable_dataset = my_iterable_dataset.filter(filter_fn)\nmy_iterable_dataset = my_iterable_dataset.map(process_fn_2)",What is the difference between `Dataset.map` and `IterableDataset.map`?,"`Dataset.map` processes the entire dataset immediately, while `IterableDataset.map` applies the function on-the-fly and only when iterating over the dataset.",huggingface/datasets/blob/main/docs/source/about_mapstyle_vs_iterable.mdx
1,"## 描述性内容\n\n在前面的示例中，您可能已经注意到 Interface 构造函数中的 `title=` 和 `description=` 关键字参数，帮助用户了解您的应用程序。\n\nInterface 构造函数中有三个参数用于指定此内容应放置在哪里：\n\n- `title`：接受文本，并可以将其显示在界面的顶部，也将成为页面标题。\n- `description`：接受文本、Markdown 或 HTML，并将其放置在标题正下方。\n- `article`：也接受文本、Markdown 或 HTML，并将其放置在界面下方。\n\n![annotated](/assets/guides/annotated.png)\n\n如果您使用的是 `Blocks` API，则可以 with `gr.Markdown(...)` 或 `gr.HTML(...)` 组件在任何位置插入文本、Markdown 或 HTML，其中描述性内容位于 `Component` 构造函数内部。\n\n另一个有用的关键字参数是 `label=`，它存在于每个 `Component` 中。这修改了每个 `Component` 顶部的标签文本。还可以为诸如 `Textbox` 或 `Radio` 之类的表单元素添加 `info=` 关键字参数，以提供有关其用法的进一步信息。\n\n```python\ngr.Number(label='年龄', info='以年为单位，必须大于0')\n```\n\n## 旗标\n\n默认情况下，""Interface"" 将有一个 ""Flag"" 按钮。当用户测试您的 `Interface` 时，如果看到有趣的输出，例如错误或意外的模型行为，他们可以将输入标记为您进行查看。在由 `Interface` 构造函数的 `flagging_dir=` 参数提供的目录中，将记录标记的输入到一个 CSV 文件中。如果界面涉及文件数据，例如图像和音频组件，将创建文件夹来存储这些标记的数据。\n\n例如，对于上面显示的计算器界面，我们将在下面的旗标目录中存储标记的数据：\n\n```directory\n+-- calculator.py\n+-- flagged/\n| +-- logs.csv\n```\n\n_flagged/logs.csv_","What is the default behavior of the ""Interface"" in the Blocks API?","The default behavior of the ""Interface"" is to display a ""Flag"" button.",gradio-app/gradio/blob/main/guides/cn/01_getting-started/02_key-features.md


### **1.3. Thiết lập các tác nhân đánh giá**

Các câu hỏi do tác nhân trước đó tạo ra có thể có nhiều lỗi: chúng ta nên kiểm tra chất lượng trước khi xác nhận các câu hỏi này.

Do đó, chúng ta xây dựng các tác nhân đánh giá sẽ xếp hạng từng câu hỏi dựa trên một số tiêu chí, được nêu trong [bài báo này](https://huggingface.co/papers/2312.10003):
- **Tính xác thực:** câu hỏi có thể được trả lời từ ngữ cảnh đã cho không?

- **Tính phù hợp:** câu hỏi có phù hợp với người dùng không? Ví dụ, `"Ngày phát hành transformers 4.29.1 là khi nào?"` không phù hợp với những người thực hành ML.

Một trường hợp lỗi cuối cùng mà chúng tôi nhận thấy là khi một hàm được thiết kế riêng cho bối cảnh cụ thể nơi câu hỏi được tạo ra, nhưng bản thân nó lại không thể giải mã được, chẳng hạn như `"Tên của hàm được sử dụng trong hướng dẫn này là gì?"`.
Chúng tôi cũng xây dựng một tác nhân phê bình cho các tiêu chí này:
- **Độc lập**: câu hỏi có dễ hiểu khi không có bất kỳ ngữ cảnh nào, đối với người có kiến ​​thức chuyên môn/truy cập Internet không? Ngược lại với điều này là câu hỏi `Hàm nào được sử dụng trong bài viết này?` đối với một câu hỏi được tạo ra từ một bài viết blog cụ thể.

Chúng tôi chấm điểm các hàm một cách có hệ thống bằng tất cả các tác nhân này, và bất cứ khi nào điểm số quá thấp đối với bất kỳ tác nhân nào, chúng tôi sẽ loại bỏ câu hỏi đó khỏi tập dữ liệu đánh giá của mình.

💡 ___Khi yêu cầu các tác nhân đưa ra điểm số, trước tiên chúng tôi yêu cầu chúng đưa ra lý do. Điều này sẽ giúp chúng tôi xác minh điểm số, nhưng quan trọng hơn, việc yêu cầu chúng đưa ra lý do trước tiên sẽ cung cấp cho mô hình nhiều token hơn để suy nghĩ và xây dựng câu trả lời trước khi tóm tắt nó thành một token điểm số duy nhất.___

Bây giờ chúng tôi xây dựng và chạy các tác nhân phê bình này.

In [36]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [37]:
import re

def parse_llm_judge_output(text):
    score_match = re.search(r"Total rating:\s*([1-5])", text, re.IGNORECASE)
    eval_match = re.search(
        r"Evaluation:\s*(.*?)(?:\n\s*Total rating:|$)",
        text,
        re.IGNORECASE | re.DOTALL
    )

    if not score_match or not eval_match:
        raise ValueError("Cannot parse judge output")

    score = int(score_match.group(1))
    evaluation = eval_match.group(1).strip()

    return score, evaluation


In [38]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
      for criterion, evaluation in evaluations.items():
        score, eval_text = parse_llm_judge_output(evaluation)

        output.update({
            f"{criterion}_score": score,
            f"{criterion}_eval": eval_text,
        })
    except Exception:
      continue


Generating critique for each QA couple...


  0%|          | 0/3 [00:00<?, ?it/s]

Giờ chúng ta hãy lọc ra những câu hỏi không hay dựa trên điểm số của hệ thống đánh giá:

In [43]:
import pandas as pd

pd.set_option("display.max_colwidth", None)# không ngắt câu hỏi/ câu trả lời

generated_questions = pd.DataFrame.from_dict(outputs) # tạo DataFrame từ outputs

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

Evaluation dataset before filtering:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the difference between `Dataset.map` and `IterableDataset.map`?,"`Dataset.map` processes the entire dataset immediately, while `IterableDataset.map` applies the function on-the-fly and only when iterating over the dataset.",5,4,4
1,"What is the default behavior of the ""Interface"" in the Blocks API?","The default behavior of the ""Interface"" is to display a ""Flag"" button.",5,3,4
2,What is the default polynomial decay schedule in Keras?,It starts at 5e-5 and decays down at a constant rate until it hits zero right at the very end of training.,3,4,1


Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
0,What is the difference between `Dataset.map` and `IterableDataset.map`?,"`Dataset.map` processes the entire dataset immediately, while `IterableDataset.map` applies the function on-the-fly and only when iterating over the dataset.",5,4,4


Giờ thì bộ dữ liệu đánh giá tổng hợp của chúng ta đã hoàn chỉnh! Chúng ta có thể đánh giá các hệ thống RAG khác nhau trên bộ dữ liệu đánh giá này.

Chúng ta chỉ tạo ra một vài cặp QA ở đây để giảm thời gian và chi phí. Nhưng hãy bắt đầu phần tiếp theo bằng cách tải một bộ dữ liệu đã được tạo sẵn:

In [ ]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

README.md:   0%|          | 0.00/893 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65 [00:00<?, ? examples/s]

# 2. Build our RAG System

### 2.1. Preprocessing documents to build our vector database

- In this part, __we split the documents from our knowledge base into smaller chunks__: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.

Many options exist for text splitting:
- split every `n` words / characters, but this has the risk of cutting in half paragraphs or even sentences
- split after `n` words / character, but only on sentence boundaries
- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).

To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.

[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

> In the following, we use Langchain's `RecursiveCharacterTextSplitter`.

💡 _To measure chunk length in our Text Splitter, our length function will not be the count of characters, but the count of tokens in the tokenized text: indeed, for subsequent embedder that processes token, measuring length in tokens is more relevant and empirically performs better._

In [ ]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### 2.2. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### 2.3. Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"
HF_API_TOKEN = ""

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    huggingfacehub_api_token=HF_API_TOKEN,
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM


def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = llm(final_prompt)

    return answer, relevant_docs

# 3. Benchmarking the RAG system

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evaluation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on Answer Correctness since it is the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [ ]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = ""

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, openai_api_key=OPENAI_API_KEY)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 Let's run the tests and evaluate answers!👇

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

### Inspect results

In [ ]:
import glob

outputs = []
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)

In [ ]:
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

In [ ]:
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()
average_scores.sort_values()

settings
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:False_reader-model:zephyr-7b-beta.json       0.884328
./output/rag_chunk:200_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader-model:zephyr-7b-beta.json    0.906716
./output/rag_chunk:200_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader-model:zephyr-7b-beta.json     0.906716
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:mixtral.json               0.906716
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:zephyr-7b-beta.json        0.921642
./output/rag_chunk:200_embeddings:thenlper~gte-small_rerank:True_reader-model:mixtral0.json              0.947761
Name: eval_score_GPT4, dtype: float64

## Example results

Let us load the results that I obtained by tweaking the different options available in this notebook.
For more detail on why these options could work or not, see the notebook on [advanced_RAG](advanced_rag).

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ ___There is no single good recipe: you should try several different directions when tuning your RAG systems.___


In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_settings_accuracy.png" height="500" width="800">

As you can see, these had varying impact on performance. In particular, tuning the chunk size is both easy and very impactful.

But this is our case: your results could be very different: now that you have a robust evaluation pipeline, you can set on to explore other options! 🗺️

Lưu lại Code và các file kết quả (Thực hiện khi muốn lưu)
Sau khi bạn đã chạy xong các code đánh giá RAG bên dưới, bạn muốn lưu lại toàn bộ file Notebook này và các file kết quả (như các file .json trong thư mục output), hãy thêm 1 ô code ở DƯỚI CÙNG của Colab:

In [ ]:
# 1. Sao chép các file kết quả từ thư mục output của Colab vào thư mục RAG để đẩy lên Git
!cp -r /content/output /content/RAG/

# 2. Thực hiện đẩy lên GitHub
!git add .
!git commit -m "Lưu trữ code và kết quả đánh giá RAG từ Colab"
!git push origin main

cp: cannot stat '/content/output': No such file or directory
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/TranQuocHuy83/RAG.git'
